<font color="#ffb142">Reading Comprehension Overview<br><font color="#636e72">Venezuela, officially the Bolivarian Republic of Venezuela (Spanish: República Bolivariana de Venezuela),[12] is a country on the northern coast of South America, consisting of a continental landmass and many islands and islets in the Caribbean Sea. It has a territorial extension of 916,445 km2 (353,841 sq mi), and the population of Venezuela was estimated at 28 million in 2019.[7] The capital and largest urban agglomeration is the city of Caracas.<br>


*   How many people live in Venezuela?

*   Q&A Response: <br>
28 million

<font color="#fd79a8">Pre-Trained Transformer Model - ALBERT. <font color="#636e72">It is said that ALBERT has shorter training and inference times than BERT. We would be fine tuning ALBERT for our Q&A task. <br>
<font color="#fd79a8"> Transformer Library: Huggings Face





<font color="#ffb142">Check GPU Settings

In [ ]:
!nvidia-smi

Fri Jan  6 09:59:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    31W /  70W |    312MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


<font color="#ffb142"> Source Hugging Face Library from Github<br>Git checkout: Switch branches or restore working tree files<br>[Git Hub Repo](https://github.com/huggingface/transformers)

In [ ]:
!git clone https://github.com/huggingface/transformers \  
!cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770 

fatal: destination path ' ' already exists and is not an empty directory.
/bin/bash: line 0: cd: transformers: No such file or directory


In [ ]:
!pip install ./transformers/
!pip install tensorboardX
!pip install transformers
!pip install sentencepiece

ERROR: Invalid requirement: './transformers/'
Hint: It looks like a path. File './transformers/' does not exist.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<font color="#ffb142"> Get Training Data SQuAD 2.0 <br>[SQuAD Resource](https://rajpurkar.github.io/SQuAD-explorer/)

In [ ]:
!mkdir dataset \
&& cd dataset \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2023-01-06 09:48:09--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   256MB/s    in 0.2s    

2023-01-06 09:48:10 (256 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2023-01-06 09:48:10--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

<font color="#ffb142">  ALBERT: Train Model <br>[Find Model Name](https://huggingface.co/transformers/pretrained_models.html) <br> [ALBERT Parameters](https://huggingface.co/transformers/model_doc/bert.html)

In [ ]:
!export SQUAD_DIR=/content/dataset \
&& python transformers/examples/run_squad_trainer.py \
  --model_type albert \
  --model_name_or_path albert-base-v2 \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file $SQUAD_DIR/train-v2.0.json \
  --predict_file $SQUAD_DIR/dev-v2.0.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /content/model_output \
  --save_steps 1000 \
  --threads 2 \
  --version_2_with_negative 


python3: can't open file 'transformers/examples/run_squad_trainer.py': [Errno 2] No such file or directory



<font color="#fd79a8"><br> [SentencePiece](https://github.com/google/sentencepiece

<font color="#ffb142">model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

In [ ]:
import os
import torch
import time 

from torch.utils.data import DataLoader 

from transformers import (
    AlbertConfig, 
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample
from transformers.data.metrics.squad_metrics import compute_predictions_logits

use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""


<font color="#fd79a8"> Configure QA Model

In [ ]:
#no of predictions we want, per question
n_best_size = 1  
max_answer_length = 30
do_lower_case = True 
#an unanswerable question will be null
null_score_diff_threshold = 0.0 

<font color="#ffb142"> Setup Model & Tensor Attributes

tensor.detach() : returns new tensor
and then we present it in form of list

In [ ]:
def to_list(tensor):
  return tensor.detach().cpu().tolist()

In [ ]:
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer
)
config = config_class.from_pretrained(model_name_or_path)

tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True)

model = model_class.from_pretrained(model_name_or_path, config=config)

TypeError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
processor = SquadV2Processor()

<font color="#ffb142">class SquadExample:<br>
    """<br>
    <font color="#fd79a8"> A single training/test example for the Squad dataset, as loaded from disk.
    Args:<br>
        qas_id: The example's unique identifier<br>
        question_text: The question string<br>
        context_text: The context string<br>
        answer_text: The answer string<br>
        start_position_character: The character position of the start of the <br>answer<br>
        title: The title of the example<br>
        answers: None by default, this is used during evaluation. Holds answers as well as their start positions.<br>
        is_impossible: False by default, set to True if the example has no possible answer.<br>
    """
    [Pytorch Documentation](https://pytorch.org/docs/stable/data.html)<br> [Model Outputs Documentation](https://huggingface.co/transformers/main_classes/output.html)

In [ ]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

#Configuring A Question Answer Model
# Config
#The number of predictions given per question.
n_best_size = 1
#The maximum token length of an answer that can be generated.
max_answer_length = 30
do_lower_case = True
#If (null_score - best_non_null) is greater than the threshold predict null.
null_score_diff_threshold = 0.0

#Returns a new Tensor
def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

#tensor attributes 
#A torch.device is an object representing the device on which a torch.Tensor is or will be allocated
#the Tensor.device property allows a torch.Tensor device to be accessed
#The torch.device contains a device type either ('cpu' or 'cuda') 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

##<font color="#ffb142"> Try out our Q&A System!!!!!!!!

In [ ]:
# context = """Venezuela, officially the Bolivarian Republic of Venezuela is a country on the northern coast of South America, consisting of a continental landmass and many islands and islets in the Caribbean Sea. It has a territorial extension of 916,445 km2 (353,841 sq mi), and the population of Venezuela was estimated at 28 million in 2019.[7] The capital and largest urban agglomeration is the city of Caracas."""
# questions = ["How many people live in venezuela"]

# context = """New Zealand (Māori: Aotearoa) is a sovereign island country in the southwestern Pacific Ocean. It has a total land area of 268,000 square kilometres (103,500 sq mi), and a population of 4.9 million. New Zealand's capital city is Wellington, and its most populous city is Auckland."""
# questions = ["How many people live in New Zealand?",
#              "What's the largest city?" ]

context = """Jennifer Lynn Lopez[2] (born July 24, 1969), also known by her nickname J.Lo, is an American singer, actress and dancer. In 1991, Lopez began appearing as a Fly Girl dancer on In Living Color, where she remained a regular until she decided to pursue an acting career in 1993. For her first leading role in the 1997 Selena biopic of the same name, Lopez became the first Latin actress to earn over US$1 million for a film. She went on to star in Anaconda (1997) and Out of Sight (1998), and has subsequently established herself as one of the highest-paid Latin actresses worldwide as of 2017.[3] Lopez is considered a pop culture icon, and is often described as a triple threat entertainer.[4][5][6]

Lopez ventured into the music industry with her debut studio album On the 6 (1999), which helped propel the Latin pop movement in American music. With the simultaneous release of her second studio album J.Lo and her romantic comedy The Wedding Planner in 2001, Lopez became the first woman to have a number-one album and film in the same week. Her 2002 release, J to tha L–O! The Remixes, became the first remix album in history to debut atop the US Billboard 200. Later that year, she released her third studio album This Is Me... Then and starred in the film Maid in Manhattan."""
questions = ["What is the nickname of Jennifer Lopez?"]
         


predictions = run_prediction(questions, context)

for key in predictions.keys():
  print(predictions[key])


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10381.94it/s]


J.Lo,
